In [14]:
import pandas as pd

df = pd.read_csv("efrs.csv")

In [ ]:
# First, read the data

nhs = pd.read_csv("~/Downloads/NHS consumption data by gender and age - Main.csv")

# Clean age bounds

def get_age_bounds(age_group: str):
    if age_group == "0 years":
        return 0, 1
    if age_group == "95 years or older":
        return 95, 120
    
    if "-" in age_group:
        lower, upper = age_group[:5].split("-")
        lower = int(lower.strip())
        upper = int(upper.strip())
        return lower, upper + 1

nhs["Lower age"] = nhs["Age group"].apply(lambda x: get_age_bounds(x)[0])
nhs["Upper age"] = nhs["Age group"].apply(lambda x: get_age_bounds(x)[1])

nhs = nhs.drop(columns=["Age group"])

index_cols = ["Lower age", "Upper age", "Gender", "Service"]

# Get counts and total costs as columns
nhs = nhs.pivot(
    index=index_cols,
    columns="Metric",
    values="Total",
)

# Roll 80+ into 80-85

nhs = nhs.reset_index()

over_80_values = (nhs[nhs["Lower age"] == 80].set_index(["Gender", "Service"]) + nhs[nhs["Lower age"] > 80].groupby(["Gender", "Service"]).sum()).reset_index()

nhs[nhs["Lower age"] == 80][["Activity Count", "Total Cost"]] = over_80_values[["Activity Count", "Total Cost"]]
nhs = nhs[nhs["Lower age"] <= 80]
nhs[nhs["Lower age"] == 80]["Upper age"] = 120

nhs["Spending per unit"] = nhs["Total Cost"] / nhs["Activity Count"]

# Now add total number in demographic groups using PE

nhs["Total people"] = np.ones_like(nhs["Total Cost"])

for i in range(len(nhs)):
    row = nhs.iloc[i]
    count = df[df.age__2022.between(row["Lower age"], row["Upper age"] - 1)][df.gender__2022 == row.Gender.upper()].household_weight__2022.sum()
    nhs.loc[i, "Total people"] = count

nhs["Per-person average units"] = nhs["Activity Count"] / nhs["Total people"]
nhs["Per-person average spending"] = nhs["Total Cost"] / nhs["Total people"]
indirect_cost_adjustment_factor = 124e9 / nhs["Total Cost"].sum() # 124 billion is the NHS budget from 2019/20, the same year as the data
# https://obr.uk/box/other-cost-pressures-in-the-health-sector/ <- points to OECD estimates of 1.7% annual increase in cost pressures that are not demographic
uprating = 1.017 ** (2022 - 2019)

nhs["Per-person average spending"] *= indirect_cost_adjustment_factor * uprating

In [170]:
nhs

Metric,Lower age,Upper age,Gender,Service,Activity Count,Total Cost,Spending per unit,Total people,Per-person average units,Per-person average spending
0,0,1,Female,AE,212079.0,3.014877e+07,142.158208,3.476480e+05,0.610039,304.450208
1,0,1,Female,APC,179975.0,2.497774e+08,1387.844784,3.476480e+05,0.517693,2522.317458
2,0,1,Female,OP,325441.0,5.887270e+07,180.901309,3.476480e+05,0.936122,594.512022
3,0,1,Male,AE,266383.0,3.859737e+07,144.894269,3.843700e+05,0.693038,352.528823
4,0,1,Male,APC,231010.0,3.242232e+08,1403.502835,3.843700e+05,0.601009,2961.290351
...,...,...,...,...,...,...,...,...,...,...
103,80,85,Female,APC,701215.0,1.349578e+09,1924.628434,1.813444e+06,0.386676,2612.644661
104,80,85,Female,OP,2162918.0,2.932964e+08,135.602179,1.813444e+06,1.192713,567.791613
105,80,85,Male,AE,323601.0,9.005399e+07,278.287104,1.350678e+06,0.239584,234.065557
106,80,85,Male,APC,679676.0,1.274705e+09,1875.459773,1.350678e+06,0.503211,3313.174141


In [179]:
from tqdm import tqdm

spending_values = []

for i in tqdm(range(len(df))):
    age = df.iloc[i].age__2022
    gender = df.iloc[i].gender__2022

    spending = nhs[nhs["Lower age"] <= age][nhs["Upper age"] > age][nhs["Gender"].str.upper() == gender]["Per-person average spending"].values[0]

    spending_values.append(spending)

100%|██████████| 214308/214308 [01:59<00:00, 1796.03it/s]


In [180]:
df["nhs_spending__2022"] = spending_values

In [181]:
df.to_csv("efrs_imputed.csv", index=False)

In [176]:
sim = Microsimulation(dataset=df)

In [177]:
px.bar(sim.calculate("nhs_spending", 2025, map_to="household").groupby(sim.calculate("household_income_decile", 2025)).mean())

ValueError: Variable nhs_spending does not exist.

In [54]:
add_fonts()

In [167]:
import plotly.express as px
from policyengine_core.charts import *
from policyengine.utils.charts import add_fonts

fig = px.line(
    nhs.groupby(["Lower age", "Gender"]).sum().reset_index(),
    x="Lower age",
    color="Gender",
    y="Per-person average spending",
    color_discrete_sequence=px.colors.qualitative.T10,
).update_layout(
    title="Spending on healthcare by gender and age",
)

format_fig(fig)

In [39]:
nhs_indexed[nhs_indexed.Metric == "Total Cost"]

,,,,Metric,Total
Lower age,Upper age,Service,Gender,,
0,1,AE,Female,Total Cost,3.014877e+07
1,5,AE,Female,Total Cost,7.330343e+07
5,10,AE,Female,Total Cost,4.900315e+07
10,15,AE,Female,Total Cost,5.526120e+07
15,20,AE,Female,Total Cost,7.935531e+07
...,...,...,...,...,...
75,80,OP,Male,Total Cost,3.709828e+08
80,85,OP,Male,Total Cost,2.820061e+08
85,90,OP,Male,Total Cost,1.545597e+08
